# GDA Implementation.

Implement the Gaussian Discriminant Analysis (GDA) learning algorithm following the steps as discussed in class.

INSTRUCTION: Rename your notebook as: <br>
`firstName_LastName_Live_coding_GDA.ipynb`.

Notes: 
* Do not use any built-in functions to complete a task;
* Do not import additional libraries.

In [171]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

In [172]:
# Generate data
def generate_data():
  x, y = make_classification(n_samples= 1000, n_features=3, n_redundant=0, 
                           n_informative=3, random_state=1, 
                           n_clusters_per_class=1)
  
  return x,y

x,y= generate_data()
print(x.shape, y.shape)

(1000, 3) (1000,)


In [173]:
d=np.column_stack((y,x))
d

array([[ 0.        , -0.42600458,  0.3092346 ,  1.13238592],
       [ 0.        ,  0.23750039,  0.85236655,  1.27032566],
       [ 1.        , -1.50956177,  0.57932947, -0.58204952],
       ...,
       [ 1.        ,  0.38796174,  1.01606996, -1.499496  ],
       [ 1.        , -0.74578403,  1.56454128, -1.05700466],
       [ 0.        ,  1.08716336, -0.29150009,  0.98548405]])

In [174]:
df1 = np.random.permutation(d.shape[0])
d=d[df1]
d


array([[ 1.        , -1.94487899,  0.66319866, -3.11399407],
       [ 1.        , -1.44587789,  1.12773178, -1.70620856],
       [ 1.        , -1.95714698, -0.01712356, -3.03870697],
       ...,
       [ 0.        ,  1.04734753,  2.4150367 ,  0.93479759],
       [ 1.        , -1.92723879,  0.85583904, -1.32963721],
       [ 1.        , -1.57584111,  1.37686666,  0.06045724]])

In [175]:
def split_data(x,y, train_size= 0.8):
    
    X_train=d[0:int(train_size*x.shape[0]),1:4]
    y_train=d[0:int(train_size*x.shape[0]),0]
    X_test=d[int(train_size*x.shape[0]):,1:4]
    y_test=d[int(train_size*x.shape[0]):,0]
    return X_train,y_train,X_test,y_test

In [176]:
X_train,  y_train,X_test, y_test= split_data(x,y, train_size= 0.8)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(800, 3) (800,) (200, 3) (200,)


In [177]:
# mu = np.zeros((x.shape[1]))
# def covariance(x,mu):
#     D, N = x.shape
#     covariance = np.zeros((N, N))
#     #mu = np.mean(x, axis=0)
#     for i in range(N):
#         for j in range(N):
#             covariance[i,j]=(1/(D-1)*np.sum(x[:,i]*x[:,j]))-(D*mu[i]*mu[j])/(D-1)

#     return covariance


In [178]:
 def covariance(x, mu):
    d = x.shape[1]
    mu=np.mean(x,axis=0)
    cov = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            cov[i, j] = np.sum((x[:,i] - mu[i]) * (x[:,j] - mu[j])) / (len(x) - 1)
    return cov

In [179]:
covariance(x,mu)

array([[1.84495325, 0.02790646, 1.00137533],
       [0.02790646, 1.00170721, 0.05539176],
       [1.00137533, 0.05539176, 1.74832   ]])

In [180]:
np.cov(x,rowvar=0)

array([[1.84495325, 0.02790646, 1.00137533],
       [0.02790646, 1.00170721, 0.05539176],
       [1.00137533, 0.05539176, 1.74832   ]])

In [181]:
phi=np.zeros(2)
for i in range(2):
    phi[i]=np.mean(y==i)
phi

array([0.498, 0.502])

In [182]:
m=np.zeros((2,3))
for i in range(2):
    for j in range(3):
        ko=np.where(y==i)
        b=x[ko]
        m[i,j]=np.mean(b[:,j].T)
m


array([[ 0.99515416,  1.04188282,  0.99941748],
       [-1.01309105,  0.95696514, -0.93218425]])

In [183]:
np.mean(x[:,0])

-0.012984932798671576

In [184]:
class GDA:
  def __init__(self):
    ## set mu, phi and sigma to None
    self.mu=None
    self.phi=None
    self.sigma=None
    
  def fit(self,x,y):
    k=2 #np.unique(y).size  # Number of class.
    d=x.shape[1] # input dim
    m= x.shape[0]# Number of examples.
    
    ## Initialize mu, phi and sigma
    self.mu= np.zeros((k,d))#: kxd, i.e., each row contains an individual class mu.
    self.sigma= np.zeros((k,d,d))#: kxdxd, i.e., each row contains an individual class sigma.
    self.phi= np.zeros((k))# d-dimension

    ## START THE LEARNING: estimate mu, phi and sigma.
    for lab in range(k):
        
        self.phi[lab] = np.sum(lab==y)/m 
        self.mu[lab] = np.mean(x[lab==y], axis =0)
        self.sigma[lab] = covariance(x[lab==y], self.mu[lab])
    return self.phi,self.mu, self.sigma
            
            

  def predict_proba(self,x):
    # reshape or flatt x.
    #x= x.reshape(-1, self.mu.shape[1])
    #x=x.reshape(-1,1)
    #x=self.mu.shape[0]
    d= x.shape[0]
    #k_class = self.mu.shape[0] 
    k_class= self.mu.shape[0]  # Number of classes we have in our case it's k = 2
    probabilities = np.zeros((d, k_class))
    
    ## START THE LEARNING: estimate mu, phi and sigma.
    for lab in range(k_class):
        det_cov= np.linalg.det(self.sigma[lab])
        inv_cov=np.linalg.inv(self.sigma[lab])
        for j in range(x.shape[0]):
            first_term=1/((2*np.pi)**(d/2)*(det_cov**0.5))
            #first_term=((1/(((2*np.pi)**(d/2))*(det_cov**0.5)))
            exponential=-0.5*((x[j]-self.mu[lab]).T)@(inv_cov)@(x[j]-self.mu[lab])
            second_term=np.exp(exponential)
            probabilities[j, lab] = first_term*second_term*self.phi[lab]
    return probabilities

  def predict(self,x):
    predict=self.predict_proba(x)
    
    y = np.argmax(predict,axis = 1)
    #Predict = np.argmax(self.predict_proba(x))
    return y
    
  
  def accuracy(self, y, ypreds):
#     ypred = self.predict(y)
    result = np.mean(y == ypreds)
    return result * 100 

In [185]:
model= GDA()
model.fit(X_train,y_train)

(array([0.49875, 0.50125]),
 array([[ 0.94944287,  1.0486899 ,  1.00393755],
        [-1.02338324,  0.92544498, -0.91987664]]),
 array([[[ 0.90472612, -0.39514042, -0.05992127],
         [-0.39514042,  1.73070546,  0.10004349],
         [-0.05992127,  0.10004349,  0.03891138]],
 
        [[ 0.80620946,  0.34220246,  0.11721226],
         [ 0.34220246,  0.35338   , -0.07501384],
         [ 0.11721226, -0.07501384,  1.65402195]]]))

In [186]:
np.mean(y_train[y_train==0])

0.0

In [187]:
X_train.shape

(800, 3)

In [188]:
yproba= model.predict_proba(X_test)
yproba

array([[3.08027739e-109, 1.48333261e-080],
       [9.79548417e-240, 9.79722059e-082],
       [7.26116871e-081, 1.73380495e-084],
       [2.64153169e-080, 1.63860676e-081],
       [1.15547923e-093, 8.00344593e-081],
       [6.16375546e-081, 3.90667607e-083],
       [9.91895325e-105, 4.56432160e-081],
       [5.93897967e-081, 6.06130537e-089],
       [2.13047162e-081, 4.15288945e-098],
       [3.53364554e-086, 4.74772557e-081],
       [3.55238101e-139, 6.64023086e-081],
       [1.06248058e-147, 1.92952038e-081],
       [1.15207539e-081, 1.24440590e-083],
       [2.92463641e-080, 5.10421540e-082],
       [2.13927271e-080, 1.27506183e-081],
       [2.73806147e-080, 3.25099066e-082],
       [1.35884865e-095, 6.38301072e-081],
       [3.25462190e-080, 4.26688692e-082],
       [1.64265467e-080, 7.63542739e-088],
       [1.69864750e-103, 9.54964576e-081],
       [1.78894161e-196, 5.79675819e-082],
       [1.01144796e-080, 4.61654926e-087],
       [2.69875626e-081, 5.35054949e-081],
       [2.9

In [189]:
ypreds= model.predict(X_test)
ypreds


array([1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1])

In [190]:
model.accuracy(y_test, ypreds)

98.0

In [192]:
class LogisticRegression:
  '''
  The goal of this class is to create a LogisticRegression class, 
  that we will use as our model to classify data point into a corresponding class
  '''
  def __init__(self,lr,n_epochs):
    self.lr = lr
    self.n_epochs = n_epochs
    self.train_losses = []
    self.w = None
    self.weight = []

  def add_ones(self, x):
    ##### WRITE YOUR CODE HERE #####
    one = np.ones((x.shape[0],1))
    return np.hstack((one,x))
    #### END CODE ####

  def sigmoid(self, x):
    return 1/(1+np.exp(-x@self.w))


  def cross_entropy(self, x, y_true):
    ##### WRITE YOUR CODE HERE #####
    y_pred = self.sigmoid(x)
    loss = -np.mean(y_true*np.log(y_pred)+(1-y_true)*np.log(1-y_pred))
    return loss
    #### END CODE ####
  
  def predict_proba(self,x):  #This function will use the sigmoid function to compute the probalities
    ##### WRITE YOUR CODE HERE #####
    x= self.add_ones(x)
    proba = self.sigmoid(x)
    return proba
    #### END CODE ####

  def predict(self,x):
    ##### WRITE YOUR CODE HERE #####
    probas = self.predict_proba(x)
    output = [0 if p<0.5 else 1 for p in probas]#np.where(probas>=0.5, 1, 0)      #convert the probalities into 0 and 1 by using a treshold=0.5
    return output
    #### END CODE ####

  def fit(self,x,y):
    # Add ones to x
    x=self.add_ones(x)

    # reshape y if needed
    y=y.reshape(-1,1)

    # Initialize w to zeros vector >>> (x.shape[1])
    self.w=np.zeros((x.shape[1],1))

    for epoch in range(self.n_epochs):
      # make predictions
      ypred = self.sigmoid(x)

      #compute the gradient
      dl = (-1/x.shape[0])*(x.T@(y-ypred))

      #update rule
      self.w=self.w-self.lr*dl

      #Compute and append the training loss in a list
      loss = self.cross_entropy(x,y)
      self.train_losses.append(loss)

      if epoch%1000 == 0:
        print(f'loss for epoch {epoch}  : {loss}')

  def accuracy(self,y_true, y_pred):
    ##### WRITE YOUR CODE HERE #####
    acc = np.mean(y_true==y_pred)*100
    return acc
    #### END CODE ####

In [193]:
model =LogisticRegression(lr=0.01,n_epochs=1000)
model.fit(X_train,y_train)

loss for epoch 0  : 0.688409308099198


In [194]:
ypred_train = model.predict(X_train)
acc = model.accuracy(y_train,ypred_train)
print(f"The training accuracy is: {acc}")
print(" ")

ypred_test = model.predict(X_test)
acc = model.accuracy(y_test,ypred_test)
print(f"The test accuracy is: {acc}")

The training accuracy is: 94.625
 
The test accuracy is: 96.0
